In [ ]:
import numpy as np
from collections import Counter
from model import Board, RandomPlayer, MiniMaxPlayer
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout

In [ ]:
class Player():
    """
    A generic player
    """
    def __init__(self, marker):
        self.marker = marker


In [ ]:
def minimax(Graph):
    """
    Perform minimax from node n on a NetworkX graph G.
    Assume node n is a maximiser node.
    Return best move
    """
    maxplayer = True
    minplayer = False
    G = Graph.copy()
    G.nodes[0].update({'player': 'max'})
    # Recursive tree search
    def _minimax(G, n, player):

        # Base case, winning node found
        if G.nodes[n]['finished']:
            if G.nodes[n]['result'] == 'won':
                score = 100
            elif G.nodes[n]['result'] == 'lost':
                score = -100
            elif G.nodes[n]['result'] == 'draw':
                score = 0
            else:
                assert True == False

            G.nodes[n].update({'score': score})
            return score

        if player == maxplayer:
            bestv = -1
            for child in G.successors(n):
                v = _minimax(G, child, minplayer)
                G.nodes[child].update({'score': v, 'player': 'min'})
                bestv = max(bestv, v)
        else:
            bestv = 1
            for child in G.successors(n):
                v = _minimax(G, child, maxplayer)
                G.nodes[child].update({'score': v, 'player': 'max'})
                bestv = min(bestv, v)
        return bestv

    # Find the best first move from the given node
    # Assume given node n is a maximiser node.
    best_node = None
    bestv = -1

    for child in G.successors(0):
        v = _minimax(G, child, minplayer)
        G.nodes[child].update({'score': v, 'player': 'min'})

        if v > bestv:
            best_node = child
            bestv = v

    return G

# Set up a player and blank board

In [ ]:
# X goes first
player = Player('X')
board = Board(state=np.zeros((3,3), dtype=int))
board.draw(board.state)

# Minimax over full game space (depth=9)

In [ ]:
%%time
G = MiniMaxPlayer.search(board, player, max_depth=9)

In [ ]:
print(G.number_of_nodes())
leaf_nodes = [node for node in G.nodes() if G.in_degree(node)!=0 and G.out_degree(node)==0]
print(len(leaf_nodes))

In [ ]:
#MiniMaxPlayer.draw_graph(G, fig_size=(5, 20), node_label='player', edge_label='move')

# Do minimax over all states

In [ ]:
%%time
H = minimax(G)

In [ ]:
H.number_of_nodes()

In [ ]:
# Add best minimax move to each node
for n in H.nodes():
    scores = [(v, c['move'], H.nodes[v]['score']) for (u, v, c) in H.out_edges(n, data=True)]
    if scores:
        best_move = max(scores, key = lambda t: t[2])[1]
        H.nodes[n]['best_move'] = best_move

In [ ]:
#MiniMaxPlayer.draw_graph(H, fig_size=(5, 20), node_label='best_move', edge_label='move')

In [ ]:
# Hash all best moves:
best_moves_X = {}

for node in H.nodes(data=True):
    if node[1].get('best_move') and node[1].get('player') == 'max':
        best_moves_X[node[1]['state'].tostring()] = node[1]['best_move']

In [ ]:
len(best_moves_X.keys())

## Save minimax dictionary to disk

In [ ]:
import pickle
# ...
with open('ttt_minimax.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(best_moves_X, f, pickle.HIGHEST_PROTOCOL)